In [2]:
import gym
import numpy as np
import pandas as pd
#import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
import tensorflow
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras import layers
from keras.optimizers import Adam
from collections import deque
import random
import datetime as dt
import alpaca_trade_api as tradeapi

ModuleNotFoundError: No module named 'pandas'

In [ ]:
# Replace 'YOUR_API_KEY' and 'YOUR_SECRET_KEY' with your actual API keys
API_KEY = 'PK58SXLOXJ0MCMQBJZVT'
SECRET_KEY = 'fYlO75pUZ3GG7rYTuBvKio21wMyXVSwQn2CB6bIT'

# Initialize the Alpaca API client
api = tradeapi.REST(API_KEY, SECRET_KEY, base_url='https://paper-api.alpaca.markets', api_version='v2')

# Define the stock symbol and time frame
symbol = 'NVD'  # Replace with the stock symbol you want to fetch data for
timeframe = '1D'  # 1D for daily data, 1H for hourly, etc.

# Get historical stock data
historical_data = api.get_bars(symbol, timeframe, limit=10).df

# Print the retrieved data
print(historical_data)


Empty DataFrame
Columns: []
Index: []


In [ ]:
# Define the Alpaca stock symbol and date range
symbol = 'AAPL'
start_date = '2000-01-01'
end = dt.date.today() - dt.timedelta(days=1)
end_date = end.strftime('%Y-%m-%d')

In [ ]:
# Download historical stock data using alpaca
df = yf.download(symbol, start=start_date, end=end_date)
df

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2000-01-03,0.936384,1.004464,0.907924,0.999442,0.848323,535796800
2000-01-04,0.966518,0.987723,0.903460,0.915179,0.776801,512377600
2000-01-05,0.926339,0.987165,0.919643,0.928571,0.788168,778321600
2000-01-06,0.947545,0.955357,0.848214,0.848214,0.719961,767972800
2000-01-07,0.861607,0.901786,0.852679,0.888393,0.754065,460734400
...,...,...,...,...,...,...
2023-09-11,180.070007,180.300003,177.339996,179.360001,179.360001,58953100
2023-09-12,179.490005,180.130005,174.820007,176.300003,176.300003,90370200
2023-09-13,176.509995,177.300003,173.979996,174.210007,174.210007,84267900


In [ ]:
# Preprocess the data
df['Date'] = df.index
df.reset_index(drop=True, inplace=True)
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

In [ ]:
# Create a function to calculate technical indicators (e.g., moving averages)
def calculate_technical_indicators(df):
    df['SMA_50'] = df['Close'].rolling(window=50).mean()
    df['SMA_200'] = df['Close'].rolling(window=200).mean()
    return df

In [ ]:
df = calculate_technical_indicators(df)

In [ ]:
# Define the state size and action space
state_size = 5  # Number of historical prices to consider
action_size = 3  # Buy, Hold, Sell

In [ ]:
# Create a DQN model using Keras
model = Sequential()
model.add(Dense(24, input_dim=state_size, activation='relu'))
model.add(Dense(24, activation='relu'))
model.add(Dense(action_size, activation='linear'))
model.compile(loss='mse', optimizer='adam')

In [ ]:
# Define hyperparameters
gamma = 0.95  # Discount factor for future rewards
epsilon = 1.0  # Exploration rate
epsilon_min = 0.01  # Minimum exploration rate
epsilon_decay = 0.995  # Decay rate for exploration

In [ ]:
# Create a replay buffer to store past experiences
replay_buffer = deque(maxlen=1000)

In [ ]:
# Define a function to select an action (Buy, Hold, or Sell) using epsilon-greedy policy
def select_action(state):
    if np.random.rand() <= epsilon:
        return random.randrange(action_size)
    q_values = model.predict(state)
    return np.argmax(q_values[0])

In [ ]:
# Define a function to preprocess the state
def preprocess_state(state):
    scaler = MinMaxScaler()
    state = np.reshape(state, [1, state_size])
    state = scaler.fit_transform(state)
    return state

In [ ]:
# Define training parameters
batch_size = 32
n_episodes = 1000

for episode in range(n_episodes):
    state = df.iloc[0:state_size]['Close'].values
    state = preprocess_state(state)
    done = False
    total_reward = 0

    while not done:
        action = select_action(state)
        next_state = df.iloc[state_size:state_size + 1]['Close'].values
        next_state = preprocess_state(next_state)
        reward = 0  # Define your reward function based on trading strategy
        total_reward += reward

        replay_buffer.append((state, action, reward, next_state, done))

        if len(replay_buffer) > batch_size:
            minibatch = random.sample(replay_buffer, batch_size)
            for state, action, reward, next_state, done in minibatch:
                target = reward
                if not done:
                    target = reward + gamma * np.amax(model.predict(next_state)[0])
                target_f = model.predict(state)
                target_f[0][action] = target
                model.fit(state, target_f, epochs=1, verbose=0)

        state = next_state

        if epsilon > epsilon_min:
            epsilon *= epsilon_decay

    print(f"Episode: {episode + 1}, Total Reward: {total_reward}")

ValueError: cannot reshape array of size 1 into shape (1,5)

In [ ]:
# Save the trained model
model.save('yahoo_stock_dqn_model.h5')